In [ ]:
# in this notebook varya is trying to export the median point (from the precipitation) from HMA - regions 
# to a dataframe and then later csv file to use as a forsing for testrun of sedcas model 

# take the locations determined in a different notebook (also stored in some csv file already)
# loop through every file that is in the folder on the server:
#    find location. select data 
#    

# find that location in the HMA climate from the server 
# convert to csv 
# save 


In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import glob

from datetime import datetime


%matplotlib inline

In [4]:
# locations_pr

## workapathround (make netcdf conversion to df faster): 

### (this is an important part) 

### locations from csv and .shp file

In [2]:
# read locations file - for annual sum precipitations: 
locations_pr = pd.read_table('out/median_annual_precip_regions.csv', sep = ',',index_col =0)

# read locations file - for annial median temp:
locations_t2m = pd.read_table('out/annual_median_t2m_regions.csv', sep = ',', index_col = 0)

# read hma polygons file to a df:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
hma_df = pd.DataFrame(hma) 

### path to folders and output to where save data 

In [5]:
path_temp = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/'

# precipitations
path_precip = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/'

# total cloud cover 
path_clouds = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/'

# incoming radiation 
path_radiation = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/'


# path for output timeseries
path_output = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"


### what mountain range are we looking at? 


In [6]:
# take the n index of the "locations precipitation" table 

n = 19
print('name of mnt range:', hma_df.Name[n])

point_lat = locations_pr.latitude[n]
point_lon = locations_pr.longitude[n]

# print(datetime.now())

name of mnt range: Eastern Tien Shan


### loop: get the csv for selected point (one mnt range) for each parameter 

In [7]:
%%time

# temperatures

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_temp)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_temp, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna()#.drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    # ds_df.to_csv(path_output + '_temp_{y}.csv'.format(y=year))
    ds_df.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/' + 'timeseries_temp_{y}.csv'.format(y=year))

    
    print('year - done:', year)
    print('done at:', datetime.now())



started at: 2022-06-21 16:52:59.862544
ready to turn into df at: 2022-06-21 16:53:00.022725
year - done: 1979
done at: 2022-06-21 16:53:36.610193
ready to turn into df at: 2022-06-21 16:53:36.648649
year - done: 1980
done at: 2022-06-21 16:53:50.095908
ready to turn into df at: 2022-06-21 16:53:50.127693
year - done: 1981
done at: 2022-06-21 16:54:04.522725
ready to turn into df at: 2022-06-21 16:54:04.588292
year - done: 1982
done at: 2022-06-21 16:54:20.696882
ready to turn into df at: 2022-06-21 16:54:20.724529
year - done: 1983
done at: 2022-06-21 16:54:54.084248
ready to turn into df at: 2022-06-21 16:54:54.131078
year - done: 1984
done at: 2022-06-21 16:55:23.519231
ready to turn into df at: 2022-06-21 16:55:23.554896
year - done: 1985
done at: 2022-06-21 16:55:53.480138
ready to turn into df at: 2022-06-21 16:55:53.539037
year - done: 1986
done at: 2022-06-21 16:56:12.039652
ready to turn into df at: 2022-06-21 16:56:12.075404
year - done: 1987
done at: 2022-06-21 16:56:37.22399

In [ ]:
#### test cell to see what am i doing wrong :/ 
'''
%%time 
preciptest = xr.open_dataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/era5_total-precipitation_hourly_1979.nc')
# preciptest2 = xr.open_dataset('/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/era5_total-precipitation_hourly_1980.nc')


# ds = xr.open_dataset(os.path.join(path_temp, f), decode_coords="all")
    # select coordinates: 
ds = preciptest.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
ds_df = ds.to_dataframe().dropna().drop_duplicates()
'''



In [ ]:
# ds_df = ds_df.reset_index()#.time.values

# for x in preciptest.time.values:
    # print(x)

In [ ]:
# ds_df = ds_df.sort_values('time') 
# for x in ds_df.time.values:
    # print(x)

In [8]:
%%time

# precipitation 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_precip)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_precip, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude'])#.squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna()#.drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    # ds_df.to_csv(path_output + '_precip_{y}.csv'.format(y=year))
    ds_df.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/' + 'timeseries_precip_{y}.csv'.format(y=year))


    print('year - done:', year)
    print('done at:', datetime.now())




started at: 2022-06-21 17:11:22.060121
ready to turn into df at: 2022-06-21 17:11:22.102816
year - done: 1979
done at: 2022-06-21 17:11:38.013408
ready to turn into df at: 2022-06-21 17:11:38.028999
year - done: 1980
done at: 2022-06-21 17:12:13.880915
ready to turn into df at: 2022-06-21 17:12:13.922335
year - done: 1981
done at: 2022-06-21 17:12:26.757820
ready to turn into df at: 2022-06-21 17:12:26.787670
year - done: 1982
done at: 2022-06-21 17:12:41.137756
ready to turn into df at: 2022-06-21 17:12:41.154195
year - done: 1983
done at: 2022-06-21 17:12:55.355254
ready to turn into df at: 2022-06-21 17:12:55.378842
year - done: 1984
done at: 2022-06-21 17:13:12.240162
ready to turn into df at: 2022-06-21 17:13:12.262427
year - done: 1985
done at: 2022-06-21 17:13:46.807719
ready to turn into df at: 2022-06-21 17:13:46.843806
year - done: 1986
done at: 2022-06-21 17:14:03.101076
ready to turn into df at: 2022-06-21 17:14:03.116616
year - done: 1987
done at: 2022-06-21 17:14:18.64978

In [9]:
%%time

# clouds 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_clouds)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_clouds, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna()#.drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    # ds_df.to_csv(path_output + '_cloud_cover_{y}.csv'.format(y=year))
    ds_df.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/' + 'timeseries_clouds_{y}.csv'.format(y=year))

    print('year - done:', year)
    print('done at:', datetime.now())
    




started at: 2022-06-21 17:28:41.533251
ready to turn into df at: 2022-06-21 17:28:41.553096
year - done: 1979
done at: 2022-06-21 17:29:05.881517
ready to turn into df at: 2022-06-21 17:29:05.903719
year - done: 1980
done at: 2022-06-21 17:29:40.788166
ready to turn into df at: 2022-06-21 17:29:40.810922
year - done: 1981
done at: 2022-06-21 17:30:09.265628
ready to turn into df at: 2022-06-21 17:30:09.285756
year - done: 1982
done at: 2022-06-21 17:30:54.141939
ready to turn into df at: 2022-06-21 17:30:54.163159
year - done: 1983
done at: 2022-06-21 17:31:18.307749
ready to turn into df at: 2022-06-21 17:31:18.328430
year - done: 1984
done at: 2022-06-21 17:31:34.176919
ready to turn into df at: 2022-06-21 17:31:34.192777
year - done: 1985
done at: 2022-06-21 17:32:13.621575
ready to turn into df at: 2022-06-21 17:32:13.638189
year - done: 1986
done at: 2022-06-21 17:32:30.531513
ready to turn into df at: 2022-06-21 17:32:30.548455
year - done: 1987
done at: 2022-06-21 17:32:45.52825

In [10]:
%%time

# radiation 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_radiation)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_radiation, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna()#.drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    # ds_df.to_csv(path_output + '_SWradiation_{y}.csv'.format(y=year))
    ds_df.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/' + 'timeseries_radiation_{y}.csv'.format(y=year))


    print('year - done:', year)
    print('done at:', datetime.now())



started at: 2022-06-21 17:43:20.172774
ready to turn into df at: 2022-06-21 17:43:20.221953
year - done: 1979
done at: 2022-06-21 17:43:36.675476
ready to turn into df at: 2022-06-21 17:43:36.692035
year - done: 1980
done at: 2022-06-21 17:44:07.130942
ready to turn into df at: 2022-06-21 17:44:07.148224
year - done: 1981
done at: 2022-06-21 17:44:45.815515
ready to turn into df at: 2022-06-21 17:44:45.833107
year - done: 1982
done at: 2022-06-21 17:45:18.657007
ready to turn into df at: 2022-06-21 17:45:18.674098
year - done: 1983
done at: 2022-06-21 17:45:51.359792
ready to turn into df at: 2022-06-21 17:45:51.378224
year - done: 1984
done at: 2022-06-21 17:46:24.553061
ready to turn into df at: 2022-06-21 17:46:24.570789
year - done: 1985
done at: 2022-06-21 17:46:57.434622
ready to turn into df at: 2022-06-21 17:46:57.454674
year - done: 1986
done at: 2022-06-21 17:47:13.496709
ready to turn into df at: 2022-06-21 17:47:13.513729
year - done: 1987
done at: 2022-06-21 17:47:48.68568

### read created .csv-s and attach them together (separately for each variable) 

In [11]:
# region #1 temperatures 

# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/1_Western_Himalaya/temp/*.csv')
# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/13_Tibetan_Interior_Mountains/temp/*.csv')
files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/temp/*.csv')



df_all = []

for f in files:
    df = pd.read_csv(f, sep = ',')
    df_all.append(df)

temps_region1 = pd.concat(df_all).sort_values('time').drop('expver', axis=1).set_index('time')
# temps_region1.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/temps_1979_2020.csv')


In [12]:
# region #1 precipitation 

# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/1_Western_Himalaya/precip/*.csv')
# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/13_Tibetan_Interior_Mountains/precip/*.csv')
files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/precip/*.csv')


df_all = []

for f in files:
    df = pd.read_csv(f, sep = ',')
    df_all.append(df)

precip_region1 = pd.concat(df_all).sort_values('time').drop('expver', axis=1).set_index('time')
# precip_region1.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/precip_1979_2020.csv')



In [13]:
# region #1 clouds 

# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/1_Western_Himalaya/clouds/*.csv')
# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/13_Tibetan_Interior_Mountains/clouds/*.csv')
files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/clouds/*.csv')


df_all = []

for f in files:
    df = pd.read_csv(f, sep = ',')
    df_all.append(df)

clouds_region1 = pd.concat(df_all).sort_values('time').drop('expver', axis=1).set_index('time')
# clouds_region1.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/clouds_1979_2020.csv')



In [14]:
# region #1 radiation 

# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/1_Western_Himalaya/radiation/*.csv')
# files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/13_Tibetan_Interior_Mountains/radiation/*.csv')
files = glob.glob('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/radiation/*.csv')


df_all = []

for f in files:
    df = pd.read_csv(f, sep = ',')
    df_all.append(df)

swradiation_region1 = pd.concat(df_all).sort_values('time').drop('expver', axis=1).set_index('time')
# swradiation_region1.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/SWradiation_1979_2020.csv')



### merge all variables together using 'time' as key 


In [24]:
# sample climate data - how it all is supposed to look like:

sampleclimate = pd.read_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/1_Western_Himalaya/sample_climate.met', sep = '\t')
# sampleclimate

In [15]:
# read all 
path = '/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan'

temps = pd.read_csv(path + '/temps_1979_2020.csv', index_col = 0)
precip = pd.read_csv(path + '/precip_1979_2020.csv', index_col = 0)
clouds = pd.read_csv(path + '/clouds_1979_2020.csv', index_col = 0)
radiation = pd.read_csv(path + '/SWradiation_1979_2020.csv', index_col = 0)

# slice data so it starts and ends at the same time
start = '1980-01-01 00:00:00' 
end = '2019-01-01 00:00:00'

temps = temps.loc[start:end,:]
precip = precip.loc[start:end,:]
clouds = clouds.loc[start:end,:]
radiation = radiation.loc[start:end,:]

In [16]:
print('temps length:', len(temps)) 
print('precip length:', len(precip)) 
print('clouds length:', len(clouds)) 
print('radiation length:', len(radiation))


temps length: 341881
precip length: 341881
clouds length: 341881
radiation length: 341881


In [18]:
# clouds = clouds.reset_index().drop_duplicates()
# clouds = clouds.set_index('time')

In [19]:
print('new length clouds:', len(clouds))

new length clouds: 341881


In [17]:
# ----- merge together:

allstuff = temps.join([precip, radiation, clouds])

# ----- convert units: 

#convert temperature K to C
allstuff['t2m']=allstuff.t2m-273.15
# precipotation m to mm 
allstuff['tp']=allstuff.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
allstuff['ssrd'] = allstuff.ssrd / 3600


In [19]:
# allstuff.head()
# allstuff[1000:1500].t2m.plot()

In [20]:
# rename columns: 

allstuff = allstuff.reset_index()

In [21]:
allstuff = allstuff.rename(columns={'time': 'D', 't2m':'Ta', 'tp':'Pr', 'tcc':'N', 'ssrd':'Rsw'})


In [22]:
allstuff.head()

,D,Ta,Pr,Rsw,N
0,1980-01-01 00:00:00,-12.92365,0.031281,0.000000,0.842461
1,1980-01-01 01:00:00,-14.07618,0.022126,0.000000,0.782384
2,1980-01-01 02:00:00,-14.28500,0.015259,0.000000,0.743381
3,1980-01-01 03:00:00,-14.21647,0.012208,30.036458,0.745823
4,1980-01-01 04:00:00,-13.98212,0.009155,124.522639,0.621427


In [25]:
sampleclimate.head()

,D,Pr,Ta,Rsw,N
0,1999-10-01 00:00:00,0.0,5.6,1.0,NaN
1,1999-10-01 01:00:00,0.0,5.5,2.0,NaN
2,1999-10-01 02:00:00,0.0,5.7,1.0,NaN
3,1999-10-01 03:00:00,0.4,5.7,0.0,NaN
4,1999-10-01 04:00:00,0.1,5.7,0.0,NaN


### save output! 

In [26]:
# allstuff.to_csv('/Users/varyabazilova/Desktop/hma_regions/out/timeseries/19_Eastern_Tien_Shan/climate.met', sep = '\t', index = False)